In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
import sys
import time

# --- 1. Cài đặt ban đầu ---
FILE_PATH = '../data1/paysim.csv' 
OUTPUT_FOLDER = 'processed_data' 

# --- 2. Hàm tạo đặc trưng (Feature Engineering) ---
def feature_engineering(df):
    """Tạo thêm các trường mới để mô hình học tốt hơn."""
    print("Bắt đầu feature engineering...")
    
    # 1. Đặc trưng lỗi số dư
    df['deltaBalanceOrig'] = df['oldbalanceOrg'] - df['newbalanceOrig']
    df['deltaBalanceDest'] = df['newbalanceDest'] - df['oldbalanceDest']
    df['errorBalanceOrig'] = df['deltaBalanceOrig'] - df['amount']
    df['errorBalanceDest'] = df['deltaBalanceDest'] - df['amount']
    
    # 2. Đặc trưng "Rút cạn tài khoản"
    df['isOrigEmptyAfterTx'] = (df['newbalanceOrig'] == 0).astype(int)

    # 3. Đặc trưng "Người nhận là Khách hàng (C)"
    # (Phải xử lý trường hợp nameDest có thể bị null - nếu có)
    df['isDestCustomer'] = df['nameDest'].str.startswith('C').fillna(0).astype(int)

    # 4. One-hot encoding cho cột 'type'
    df = pd.get_dummies(df, columns=['type'], prefix='type', dtype=int)
    
    print("Hoàn tất feature engineering.")
    return df

# --- 3. Quy trình xử lý chính ---
def preprocess_data(file_path):
    
    # --- 2. Tải dữ liệu ---
    print(f"Đang tải dữ liệu từ {file_path}...")
    start_time = time.time()
    try:
        df = pd.read_csv(file_path)
        df = df.sort_values(by='step').reset_index(drop=True) 
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file {file_path}. Vui lòng kiểm tra lại đường dẫn.")
        return None
    
    print(f"Tải xong sau {time.time() - start_time:.2f} giây.")
    print(f"Kích thước bộ dữ liệu GỐC (100%): {len(df)}")
    
    # ================================================
    # ⭐️ BƯỚC THÊM MỚI: KIỂM TRA NULL ⭐️
    # ================================================
    print("\n--- 3. Kiểm tra dữ liệu Null (Thiếu) ---")
    null_counts = df.isnull().sum()
    total_nulls = null_counts.sum()
    
    if total_nulls > 0:
        print(f"CẢNH BÁO: Phát hiện {total_nulls} giá trị null!")
        # In ra các cột có chứa null và số lượng
        print(null_counts[null_counts > 0]) 
        
        # Tạm thời, một chiến lược đơn giản là xóa các hàng bị null
        # (Trong thực tế, bạn có thể cần điền (fill) dữ liệu thay vì xóa)
        print(f"Đang xóa {total_nulls} hàng có chứa giá trị null...")
        df_old_len = len(df)
        df = df.dropna()
        print(f"Đã xóa {df_old_len - len(df)} hàng. Kích thước mới: {len(df)}")
    else:
        print("Không tìm thấy giá trị null nào. Rất tốt.")
    # ================================================
    
    # --- 4. Feature Engineering & One-Hot Encoding ---
    # (Tên bước được đánh số lại)
    df = feature_engineering(df)
    
    # --- 5. Chọn các cột đặc trưng ---
    print("\n--- 5. Chọn đặc trưng (Features) ---")
    drop_cols = ['step', 'nameOrig', 'nameDest', 'isFraud', 'isFlaggedFraud', 
                 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
            
    feature_cols = [col for col in df.columns if col not in drop_cols]
    
    print(f"Sử dụng {len(feature_cols)} đặc trưng (features):")
    print(feature_cols)

    # --- 6. Tách X (Đặc trưng) và y (Nhãn) ---
    X = df[feature_cols]
    y = df['isFraud'].values
    
    # --- 7. Tách Train/Test (70% Train, 30% Test) ---
    print("\n--- 7. Tách Train/Test (70/30) ---")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.3,      
        random_state=42, 
        stratify=y          
    )
    
    print(f"Tập Train: {X_train.shape[0]} mẫu (trong đó {np.sum(y_train)} là gian lận)")
    print(f"Tập Test: {X_test.shape[0]} mẫu (trong đó {np.sum(y_test)} là gian lận)")
    
    # --- 8. Chuẩn hóa (Scaling) ---
    print("\n--- 8. Chuẩn hóa (Scaling) ---")
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    
    X_train_scaled = scaler.transform(X_train).astype(np.float32)
    X_test_scaled = scaler.transform(X_test).astype(np.float32)
    
    print("\n--- HOÀN TẤT TIỀN XỬ LÝ ---")
    print(f"X_train_scaled shape: {X_train_scaled.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_test_scaled shape: {X_test_scaled.shape}")
    print(f"y_test shape: {y_test.shape}")
    
    return X_train_scaled, X_test_scaled, y_train, y_test

# --- 4. Chạy quy trình ---
if __name__ == "__main__":
    
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    
    result = preprocess_data(FILE_PATH)
    
    if result:
        X_train, X_test, y_train, y_test = result
        
        print(f"\nĐang lưu các file đã xử lý vào thư mục '{OUTPUT_FOLDER}'...")
        
        np.save(os.path.join(OUTPUT_FOLDER, 'X_train.npy'), X_train)
        np.save(os.path.join(OUTPUT_FOLDER, 'y_train.npy'), y_train)
        np.save(os.path.join(OUTPUT_FOLDER, 'X_test.npy'), X_test)
        np.save(os.path.join(OUTPUT_FOLDER, 'y_test.npy'), y_test)
        
        print(f"Đã lưu thành công 4 file .npy vào thư mục: {OUTPUT_FOLDER}")
    else:
        print("Không thể lưu file do quá trình tiền xử lý thất bại.")

Đang tải dữ liệu từ ../data1/paysim.csv...
Tải xong sau 8.52 giây.
Kích thước bộ dữ liệu GỐC (100%): 6362620

--- 3. Kiểm tra dữ liệu Null (Thiếu) ---
Không tìm thấy giá trị null nào. Rất tốt.
Bắt đầu feature engineering...
Hoàn tất feature engineering.

--- 5. Chọn đặc trưng (Features) ---
Sử dụng 12 đặc trưng (features):
['amount', 'deltaBalanceOrig', 'deltaBalanceDest', 'errorBalanceOrig', 'errorBalanceDest', 'isOrigEmptyAfterTx', 'isDestCustomer', 'type_CASH_IN', 'type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT', 'type_TRANSFER']

--- 7. Tách Train/Test (70/30) ---
Tập Train: 4453834 mẫu (trong đó 5749 là gian lận)
Tập Test: 1908786 mẫu (trong đó 2464 là gian lận)

--- 8. Chuẩn hóa (Scaling) ---

--- HOÀN TẤT TIỀN XỬ LÝ ---
X_train_scaled shape: (4453834, 12)
y_train shape: (4453834,)
X_test_scaled shape: (1908786, 12)
y_test shape: (1908786,)

Đang lưu các file đã xử lý vào thư mục 'processed_data'...
Đã lưu thành công 4 file .npy vào thư mục: processed_data
